In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import joblib


In [ ]:
train_data = pd.read_csv("/content/train.csv")

In [ ]:
train_data.shape

(891, 12)

In [ ]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
duplicates_count = train_data.duplicated().sum()
print("Number of duplicate rows:", duplicates_count)

Number of duplicate rows: 0


In [ ]:
missing_counts = train_data.isnull().sum()
print(missing_counts)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Data preprocessing

In [ ]:
train_data["Embarked"] = train_data["Embarked"].fillna(train_data["Embarked"].mode()[0])

In [ ]:
train_data["Sex"] = train_data["Sex"].map({"male": 0, "female": 1})

In [ ]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())


In [ ]:
train_data = train_data.drop(columns=["Cabin"])

In [ ]:
missing_counts = train_data.isnull().sum()
print(missing_counts)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [ ]:
train_data["Embarked"] = train_data["Embarked"].map({"S": 0, "C": 1, "Q": 2})


In [ ]:
train_data = train_data.drop(columns=["Name", "Ticket"])


Feature Engineering

In [ ]:
train_data["FamilySize"] = train_data["SibSp"] + train_data["Parch"] + 1


In [ ]:
train_data["IsAlone"] = (train_data["FamilySize"] == 1).astype(int)


Model Training

In [ ]:
train_data.shape

(891, 11)

In [ ]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare",
            "FamilySize", "IsAlone", "Embarked"]

X = train_data[features]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_val)
rf_acc = accuracy_score(y_val, rf_pred)

print("RandomForest Accuracy:", round(rf_acc * 100, 2), "%")


RandomForest Accuracy: 81.56 %


In [ ]:
!pip install xgboost


In [ ]:

xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_val)
xgb_acc = accuracy_score(y_val, xgb_pred)

print("XGBoost Accuracy:", round(xgb_acc * 100, 2), "%")


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [15:25:36] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Accuracy: 83.8 %


In [ ]:
if xgb_acc > rf_acc:
    best_model = xgb_model
    print("Best model: XGBoost")
else:
    best_model = rf_model
    print("Best model: RandomForest")


Best model: XGBoost


In [ ]:
joblib.dump(best_model, "best_titanic_model.pkl")
print("Saved best model to best_titanic_model.pkl")


Saved best model to best_titanic_model.pkl


Test Data


In [ ]:
test_data = pd.read_csv("/content/test.csv")

In [ ]:
test_data = test_data.drop(columns=["Cabin"])
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())
test_data["Fare"] = test_data["Fare"].fillna(test_data["Fare"].median())
test_data["Embarked"] = test_data["Embarked"].fillna(test_data["Embarked"].mode()[0])
test_data["Sex"] = test_data["Sex"].map({"male": 0, "female": 1})
test_data["Embarked"] = test_data["Embarked"].map({"S": 0, "C": 1, "Q": 2})
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"] + 1
test_data["IsAlone"] = (test_data["FamilySize"] == 1).astype(int)
test_data = test_data.drop(columns=["Name", "Ticket"])


In [ ]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare",
            "FamilySize", "IsAlone", "Embarked"]

X_test = test_data[features]


In [ ]:

best_model = joblib.load("best_titanic_model.pkl")


In [ ]:
test_predictions = best_model.predict(X_test)


In [ ]:
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})

submission.to_csv("submission.csv", index=False)
print("submission.csv created successfully!")


submission.csv created successfully!


In [ ]:
submission.head()
print("Rows:", len(submission))


Rows: 418


In [ ]:
test_predictions[:10]


array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [ ]:
preview = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": test_predictions
})

preview.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [ ]:
preview.to_csv("submission.csv", index=False)
